In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import pyplot
import requests
from bs4 import BeautifulSoup
from Bio.PDB import *
import os
from os import listdir
from os.path import isfile, join


# Общие функции

In [ ]:
def readfasta(filename):
    infile = open(filename, "r") #открываем файл
    cath = {}
    for line in infile:
        line = line.strip() #делаем файл в одну строчку
        if line.startswith(">"): #ищем символ который начинает с 
            fields1 = line.split("|") #разделяем нашедшуюся линию по проблеам
            fields1 = str(fields1)
            name1 = fields1[0][1:] #это то как мы определяем имя
            cath[fields1] = "" #и пустую последовательность строчную
        else:
            cath[fields1] =  cath[fields1] + line.strip() #1) создали ключ сек 2) добавили туда строчки, которые начинаются не с галочки
    #infile.close()
    #print(len(cath))
    return cath

In [ ]:
def remooveuseless(inter):
    inter1 = {}
    for ku, vu in inter.items():
        if (len(vu) <= 60):
            inter1[ku] = vu
    return inter1

In [ ]:
def printer(inter_uniq1):
    strings1 = [] 
    for key,item in inter_uniq1.items():
        strings1.append(">{} \n{}".format(key, item))
    result1 = "\n".join(strings1)
    return result1

# Построение дерева в питоне

In [ ]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import AlignIO
import pylab
import matplotlib.pyplot as plt

In [ ]:
def align(in_file, out_file):
    muscle_exe = r"C://tmp/muscle.exe"
    # MuscleCommandline( muscle_exe, input=in_file, out=out_file)
    # muscle_cline = MuscleCommandline(muscle_exe, align=in_file, out=out_file)
    # print(muscle_cline)
    process = subprocess.run(muscle_exe + ' -align "' + in_file+ '" -output "'+out_file+'"', shell=True)
    print(process.stderr)

In [ ]:
def make_me_tree(filename):
    aln = AlignIO.read(filename, 'fasta')
    calculator = DistanceCalculator('blosum45')
    dm = calculator.get_distance(aln)
    constructor = DistanceTreeConstructor()
    tree = constructor.nj(dm)
    return tree

In [ ]:
for name in names:
    ch = readfasta(pref+"in/"+name)
    ch = remooveuseless(ch)
    f = printer(ch)
    g = open(pref+"mid/not_aligned_"+name,"w")
    g.write(f)
    g.close()
    align(pref+"mid/not_aligned_"+name,pref+"mid/aligned_"+name)
    ch = make_me_tree(pref+"mid/aligned_"+name)
    #f = printer(ch)
    Phylo.write(ch, pref+"out/"+name[:-6]+".xml", "phyloxml")

# Работа со вторичной структурой

In [ ]:
def do_one_df(f):
    #filename = filename.replace('\','/'')
    #with open (filename) as f:
      aa_sp = []
      twostruc_sp = []
      fut_df = {}
      for line in f:
        p = line.replace(" ","")
            
        if p[0].isdigit() == True:
          line = line.split(" ")
          line = ' '.join(line).split()
              
          aa_sp.append(line[1])
          twostruc_sp.append(line[2])
      fut_df[str(aa_sp)] = twostruc_sp
      fut_sp2 = {}
      for ki, wi in fut_df.items():
        ki = str(ki)
        wi = str(wi)
        fut_sp2[ki] = wi

      c = pd.DataFrame(list(fut_sp2.items()),
                        columns=['aa', '2st'])
      c["aa"] = c["aa"].str.replace('\W','', regex=True)
      c["2st"] = c["2st"].str.replace('\W','', regex=True)
      return c

In [ ]:
sp1 = []
for element in os.listdir(r"\\wsl.localhost\Debian\home\sa\bio\weights\s4pred\output_0_40"):
    with open (os.path.join(r"\\wsl.localhost\Debian\home\sa\bio\weights\s4pred\output_0_40", element)) as f:
        c = do_one_df(f)
        sp1.append(c)
c = pd.DataFrame(np.concatenate(sp1), columns=["aa", "2st"])

In [ ]:
c = c[c['2st'].str.contains(r'H') == False]
c = c[c['2st'].str.contains(r'E') == True]


In [ ]:
po = dict(zip(c["aa"], c["2st"]))
names = {}
for ki, wi in po.items():
    for ci, vi in all.items():
        if ki == vi:
            sp = []
            sp.append(wi)
            sp.append(ki)
            names[ci] = sp
new_with_names = pd.DataFrame(names.items())

In [ ]:
df3 = pd.DataFrame(new_with_names[1].to_list(), columns=['2st','aa'])
df3.insert(0, "names", new_with_names[0])

In [ ]:
apd3 = readfasta(r"C:\Users\melib\Downloads\AMP\APD3\APD_sequence_release_09142020.fasta.txt")
v_20_40 = {}
for ki, wi in apd3.items():
    if len(wi) <=20 or (len(wi) >=35 and len(wi) <=40):
        v_20_40[ki] = wi
len(v_20_40)
printer(v_20_40)

# Аннотация на красивое дерево

читаем дерево (по id)

In [ ]:
sup_sp = []
with open(r"C:\Users\melib\Downloads\MFTI\newick_unrooted", "r") as tree:
    for line in tree:
        line = line.split("A")
        for name in line:
            sup = name[-5::]
            sup = sup.replace("(", "")
            sup = sup.replace(",", "")
            sup_sp.append(sup)
sup_sp

### распределяем по заряду/гидрофобности

тут идет парсинг apd3 для получения данных о заряде/гидрофобности/активности/откуда получен амп

In [ ]:
def net_hyt(file,sp):
   # sp=[]
#    html_doc = open(r"C:\Users\melib\Downloads\AMP\annotate\hydro\hydro3.html", encoding="utf8")
    soup = BeautifulSoup(file, 'html.parser')
    t = soup.select("input")
    t = str(t)
    for i in range(len(t)):
#    print(t[i:i+7])
        if t[i:i+7]=='value="':
         sp.append(t[i+8:i+12])
   

    return set(sp)

In [ ]:
dt={}
for file in os.listdir(r'C:\Users\melib\Downloads\AMP\annotate\net'):
    if Path(file).suffix == ".html":
         dt[file]=[]
         with open(os.path.join(r"C:\Users\melib\Downloads\AMP\annotate\net", file), 'r', encoding="utf8") as html_doc:
            #for i in rangenet):
            
               ppp = net_hyt(html_doc,dt[file])
               dt[file]=ppp
dt

In [ ]:
for file in os.listdir(r'C:\Users\melib\Downloads\AMP\annotate\hydro'):
    if Path(file).suffix == ".html":
         dt[file]=[]
         with open(os.path.join(r"C:\Users\melib\Downloads\AMP\annotate\hydro", file), 'r', encoding="utf8") as html_doc:
            #for i in rangenet):
            
               ppp = net_hyt(html_doc,dt[file])
               dt[file]=ppp
dt

In [ ]:
dt1 = {}
for id in sup_sp:
    for k, v in dt.items():
        if "net" in k and id in v:
            dt1[id+"A"]=[]
            netcharge=k[3::].replace(".html", "")
            dt1[id+"A"].append(netcharge)
    for k, v in dt.items():
        if "hydro" in k and id in v:
            #dt1[id]=[]
            hydrocharge=k[5::].replace(".html", "")
            dt1[id+"A"].append(hydrocharge)

dt1

In [ ]:
df = pd.DataFrame.from_dict(dt1, orient='index')
df.to_csv("annot_ex_shape.txt", sep = ",", header=False)

### распределяем позвы/беспы + по активности

In [ ]:
in_anim = readfasta(r"C:\Users\melib\Downloads\AMP\APD3\APD_sequence_release_09142020.fasta.txt")
invertebrates=[]
vertebrates=[]
other = []
for ki in in_anim.keys():
    ki1 = ki.split(",")
    if "invertebrates" in ki:
        invertebrates.append(ki1[0])
    else:
        if "animals" in ki:
            vertebrates.append(ki1[0])
        else:
            other.append(ki1[0])
print(invertebrates, len(invertebrates), vertebrates, len(vertebrates), other)

In [ ]:
def annot_clade_range(col1,col2,col3):
    dt = {}
    with open(r"C:\Users\melib\Downloads\AMP\gram_p_m.txt", "r") as gram_m:
         for line in gram_m:
            line = line.split(",")
        
            for el in line:
                el=el[-5:-1]
            #print(el)
                for ell in sup_sp:
                    if el in ell:
                        dt[el+"A"]=[]
                        dt[el+"A"].append("range")
                        dt[el+"A"].append(col1)
                    
                        dt[el+"A"].append("gramm negative & gramm positive")
                    #dt[el+"A"] =  + ", gramm negative & gramm positive"


    with open(r"C:\Users\melib\Downloads\AMP\gram_m.txt", "r") as gram_m:
         for line in gram_m:
            line = line.split(",")
        
            for el in line:
                el=el[-5:-1]
          #  print(el)
                for ell in sup_sp:
                    if el in ell:
                        dt[el+"A"]=[]
                        dt[el+"A"].append("range")
                        dt[el+"A"].append(col2)
                    
                        dt[el+"A"].append("gramm negative")
                    #dt[el+"A"] =  + ", gramm negative"

    with open(r"C:\Users\melib\Downloads\AMP\gram_p.txt", "r") as gram_m:
         for line in gram_m:
            line = line.split(",")
        
            for el in line:
                el=el[-5:-1]
           # print(el)
                for ell in sup_sp:
                    if el in ell:
                        dt[el+"A"]=[]
                        dt[el+"A"].append("range")
                        dt[el+"A"].append(col3)
                    
                        dt[el+"A"].append("gramm positive")
                   # dt[el+"A"] = "#ffe599" + ", gramm positive"
            
    return dt


In [ ]:
def annot_shape(symb1,s1,col1,pos1,symb2,s2,col2,pos2):
    dt = {}
    with open(r"C:\Users\melib\Downloads\AMP\APD3\APD_sequence_release_09142020.fasta.txt", "r") as gram_m:
         for line in gram_m:
            line = line.split(">")
        
      
            for ell in sup_sp:
                    for el in line:
                     if ell in el and "def" in el:
                        dt[ell+"A"]=[] #id
                        dt[ell+"A"].append(symb1) #symbol
                        dt[ell+"A"].append(s1) #size
                        dt[ell+"A"].append(col1) #color
                        dt[ell+"A"].append("1") #fill
                        dt[ell+"A"].append(pos1) #position
                   # dt[ell+"A"].append("gramm negative & gramm positive")
                     if ell in el and "cathelicidin" in el:
                        dt[ell+"A"]=[] #id
                        dt[ell+"A"].append(symb2) #symbol
                        dt[ell+"A"].append(s2) #size
                        dt[ell+"A"].append(col2) #color
                        dt[ell+"A"].append("1") #fill
                        dt[ell+"A"].append(pos2) #position
    return dt


In [ ]:
def annot_label(col1, col2, col3):
   # invertebrates
    anim_other={}
    for elem in sup_sp:
        for el in invertebrates:
            if elem in el:
                anim_other[elem+"A"]=[] #id
                anim_other[elem+"A"].append("label")
                anim_other[elem+"A"].append("node")
                anim_other[elem+"A"].append("#000000")
                anim_other[elem+"A"].append("1")
                anim_other[elem+"A"].append("normal")
                anim_other[elem+"A"].append(col1)
        for el1 in vertebrates:
            if elem in el1:
                anim_other[elem+"A"]=[] #id
                anim_other[elem+"A"].append("label")
                anim_other[elem+"A"].append("node")
                anim_other[elem+"A"].append("#000000")
                anim_other[elem+"A"].append("1")
                anim_other[elem+"A"].append("normal")
                anim_other[elem+"A"].append(col2)
        for el2 in other:
            if elem in el2:
                anim_other[elem+"A"]=[] #id
                anim_other[elem+"A"].append("label")
                anim_other[elem+"A"].append("node")
                anim_other[elem+"A"].append("#000000")
                anim_other[elem+"A"].append("1")
                anim_other[elem+"A"].append("normal")
                anim_other[elem+"A"].append(col3)
    return anim_other

In [ ]:
dt = annot_clade_range("#b4a7d6","#ffd966","#6fa8dc")
dt1 = annot_shape("3","15", "#f44336", "0.8", "1", "15", "#8fce00", "0.8")
anim_other = annot_label("#1223456", "#adbcca", "#89730a")

итоговое дерево строилось на itol

## парсинг пдб

чтобы удостовериться, что имена нигде не перепутаны

In [ ]:
def name(x):
    structure = pdb_parser.get_structure("PHA-L",x)
    counter = 1
    for polypeptide in polypeptide_builder.build_peptides(structure):
        seq = polypeptide.get_sequence()
    #print(f"Sequence: {counter}, Length: {len(seq)}")
    #print(seq)
        counter += 1
    return seq

In [ ]:
sp_name={}
for file in os.listdir(r"C:\Users\melib\Downloads\AMP\def+cath"):

   with open(os.path.join(r"C:\Users\melib\Downloads\AMP\def+cath", file), 'r') as struc:
    #print(struc)
    name1 = name(struc)
   # name1 = name(file)
    sp_name[file]=name1
sp_name

In [ ]:
cath_def = readfasta(r"C:\Users\melib\Downloads\AMP\APD3\APD_sequence_release_09142020.fasta.txt")


In [ ]:
right_na={}
for ki, wi in cath_def.items():
    for ci, vi in sp_name.items():
        if wi in vi:
            right_na[ki[-4::]]=ci
right_na

In [ ]:
for c, v in right_na.items():
 for filename in os.listdir(r"C:\Users\melib\Downloads\AMP\def+cath"):

    if filename == v:
        c = c + ".pdb"
        os.rename(os.path.join(r"C:\Users\melib\Downloads\AMP\def+cath",filename),os.path.join(r"C:\Users\melib\Downloads\AMP\def+cath\new",c))

In [ ]:
for filename in os.listdir(r"C:\Users\melib\Downloads\Telegram Desktop\final\final"):
    for i in range(196):
        #print(f'def{i}_')
        if f"def{i}_" in filename and "ranked_1" in filename:
            os.rename(os.path.join(r"C:\Users\melib\Downloads\Telegram Desktop\final\final",filename),os.path.join(r"C:\Users\melib\Downloads\AMP\def+cath","s"+str(i)+".pdb"))